# UK - Anbindung Paid Themen 

* Um was handelt es sich hier  (Kurzbeschreibung Inhalt):
- Die Paid/Organic Data Transformatation von Youtube


---
* QUELLEN:  
- datif_pz_uk_{}.03_transformed.youtube_paid_daily
- datif_pz_uk_{}.03_transformed.youtube_paid_total


* ZIEL:  
- datif_pz_uk_{}.03_transformed.youtube_paid_organic_daily
- datif_pz_uk_{}.03_transformed.youtube_paid_organic_total

---
* Versionen (aktuelle immer oben):
- 16.09.2025 Justin Stange-Heiduk: init



# 1. Initialnotebooks & Libraries

In [0]:
%run ../../common/nb_init

In [0]:
target_schema_name = "03_transformed"
target_path = "funnel"

In [0]:
from pyspark.sql.functions import col, when

## 03-transformed

### Youtube Paid Organic Daily

In [0]:
spark.sql(
    f"""
    CREATE OR REPLACE VIEW datif_pz_uk_{env}.03_transformed.youtube_paid_organic_daily AS
    SELECT
        CASE
            WHEN yo.VideoID IS NULL THEN 'Paid'
            WHEN yp.OrganicID IS NULL THEN 'Organic'
            ELSE 'Organic&Paid'
        END AS Source,
        COALESCE(yo.Date, yp.Date, 'Existiert nicht') AS Date,
        COALESCE(CAST(yo.CreatedDate AS STRING), 'Existiert nicht') AS CreatedDate,
        COALESCE(yo.VideoID, 'Existiert nicht') AS OrganicID,
        COALESCE(yo.VideoTitle, 'Existiert nicht') AS VideoTitle,
        COALESCE(yo.VideoURL, 'Existiert nicht') AS URL,
        COALESCE(yo.Tags, 'Existiert nicht') AS Tags,
        COALESCE(yo.VideoDescription, 'Existiert nicht') AS VideoDescription,
        COALESCE(CAST(yo.Views AS STRING), 0) AS All_Views,
        COALESCE(CAST(yo.Engagements AS STRING), 0) AS All_Engagements,
        COALESCE(CAST(yo.WeightedEngagements AS STRING), 0) AS All_WeightedEngagements,
        COALESCE(CAST(yo.Likes AS STRING), 0) AS All_Likes,
        COALESCE(CAST(yo.Dislikes AS STRING), 0) AS All_Dislikes,
        COALESCE(CAST(yo.Comments AS STRING), 0) AS All_Comments,
        COALESCE(CAST(yo.Shares AS STRING), 0) AS All_Shares,
        COALESCE(CAST(yo.TotalAverageViewDuration AS STRING), 0) AS All_AverageViewDuration,

        COALESCE(yp.AdID, 'Existiert nicht') AS AdID,
        COALESCE(yp.CampaignName, 'Existiert nicht') AS CampaignName,
        COALESCE(yp.AdSetName, 'Existiert nicht') AS AdSetName,
        COALESCE(yp.AdName, 'Existiert nicht') AS AdName,
        COALESCE(CAST(yp.Amount_Spend AS STRING), 0) AS Amount_Spend,
        COALESCE(yp.Video_View, 0) AS Paid_VideoView,
        COALESCE(CAST(yp.Impressions AS STRING), 0) AS Paid_Impressions,
        COALESCE(CAST(yp.Engagements AS STRING), 0) AS Paid_Engagements,

        CASE
            WHEN (yo.VideoID IS NOT NULL AND yp.OrganicID IS NOT NULL) 
                 AND (CAST(All_Views AS BIGINT) - COALESCE(Paid_VideoView, 0)) > 0
            THEN CAST(All_Views AS BIGINT) - COALESCE(Paid_VideoView, 0)
            ELSE CAST(All_Views AS BIGINT)
        END AS Organic_Views,
        CASE
            WHEN (yo.VideoID IS NOT NULL AND yp.OrganicID IS NOT NULL) 
                 AND (CAST(All_Engagements AS BIGINT) - COALESCE(Paid_Engagements, 0)) > 0
            THEN CAST(All_Engagements AS BIGINT) - COALESCE(Paid_Engagements, 0)
            ELSE CAST(All_Engagements AS BIGINT)
        END AS Organic_Engagements,
    COALESCE(CAST(yo.WeightedEngagements AS STRING), 0) AS Organic_WeightedEngagements
    FROM datif_pz_uk_{env}.03_transformed.youtube_organic_post_daily_view AS yo
    FULL OUTER JOIN datif_pz_uk_{env}.03_transformed.youtube_paid_daily AS yp
        ON yo.VideoID = yp.OrganicID
        AND yo.Date = yp.Date
    """
)


### Youtube Paid Organic Total

In [0]:
spark.sql(f"""
CREATE OR REPLACE VIEW datif_pz_uk_{env}.03_transformed.youtube_paid_organic_total AS
SELECT
    -- =============================
    -- 1. Allgemeine Metadaten
    -- =============================
    CASE
        WHEN yo.VideoID IS NULL THEN 'Paid'
        WHEN yp.OrganicID IS NULL THEN 'Organic'
        ELSE 'Organic&Paid'
    END AS Source,
    COALESCE(yo.VideoID, 'Existiert nicht') AS OrganicID,
    COALESCE(yo.CreatedDate, 'Existiert nicht') AS Organic_CreatedDate,
    COALESCE(yp.CreatedDate, 'Existiert nicht') AS Paid_CreatedDate,
    COALESCE(yo.VideoTitle, 'Existiert nicht') AS VideoTitle,
    COALESCE(yo.VideoURL, 'Existiert nicht') AS VideoURL,
    COALESCE(yo.VideoDescription, 'Existiert nicht') AS VideoDescription,
    COALESCE(yo.Tags, 'Existiert nicht') AS Tags,
    COALESCE(yo.Owner, 'Existiert nicht') AS Owner,

    -- =============================
    -- 2. Kombinierte KPIs (All_)
    -- =============================
    COALESCE(yo.TotalViews, 0) AS All_TotalViews,
    COALESCE(yo.TotalEngagements, 0) AS All_TotalEngagements,
    COALESCE(yo.TotalWeightedEngagements, 0) AS All_TotalWeightedEngagements,
    COALESCE(yo.TotalLikes, 0) AS All_TotalLikes,
    COALESCE(yo.TotalDislikes, 0) AS All_TotalDislikes,
    COALESCE(yo.TotalComments, 0) AS All_TotalComments,
    COALESCE(yo.TotalShares, 0) AS All_TotalShares,
    COALESCE(yo.TotalAverageViewDuration, 0) AS All_AverageViewDuration,

 

    -- =============================
    -- 3. Paid-Kennzahlen
    -- =============================
    COALESCE(yp.ID, 'Existiert nicht') AS AdID,
    COALESCE(yp.CampaignName, 'Existiert nicht') AS CampaignName,
    COALESCE(yp.AdName, 'Existiert nicht') AS AdName,
    COALESCE(ROUND(yp.TotalSpend, 2), 0) AS TotalSpend,
    COALESCE(yp.TotalVideoView, 0) AS Paid_TotalViews,
    COALESCE(yp.TotalImpressions, 0) AS Paid_TotalImpressions,
    COALESCE(yp.TotalEngagement, 0) AS Paid_TotalEngagements,
    COALESCE(yp.CPM, 0) AS CPM,
    COALESCE(yp.CPC, 0) AS CPC,
    COALESCE(yp.CTR, 0) AS CTR,
    COALESCE(yp.CPV, 0) AS CPV,
    COALESCE(yp.VideoViewRate, 0) AS VideoViewRate,
    COALESCE(yp.VTR, 0) AS VTR,

    -- =============================
    -- 4. Neue Themen-Spalten (aus youtube_organic_post_total)
    -- =============================
    COALESCE(yo.Themenbereich1, 'Existiert nicht') AS Themenbereich1,
    COALESCE(yo.Themenbereich2, 'Existiert nicht') AS Themenbereich2,
    COALESCE(yo.Themenbereich3, 'Existiert nicht') AS Themenbereich3,
    COALESCE(yo.Themenbereich1_Conf, 0) AS Themenbereich1_Conf,
    COALESCE(yo.Themenbereich2_Conf, 0) AS Themenbereich2_Conf,
    COALESCE(yo.Themenbereich3_Conf, 0) AS Themenbereich3_Conf,
    COALESCE(yo.Strategie2030, 0.0) AS Strategie2030,
    COALESCE(yo.FinanzierungEnergiewende, 0.0) AS FinanzierungEnergiewende,
    COALESCE(yo.EMobilitaet, 0.0) AS EMobilitaet,
    COALESCE(yo.VernetzeEnergiewelt, 0.0) AS VernetzeEnergiewelt,
    COALESCE(yo.TransformationGasnetzeWasserstoff, 0.0) AS TransformationGasnetzeWasserstoff,
    COALESCE(yo.ErneuerbareEnergien, 0.0) AS ErneuerbareEnergien,
    COALESCE(yo.DisponibleErzeugung, 0.0) AS DisponibleErzeugung,
    COALESCE(yo.IntelligenteStromnetze, 0.0) AS IntelligenteStromnetze,
    COALESCE(yo.EnBWAlsArbeitgeberIn, 0.0) AS EnBWAlsArbeitgeberIn,
    COALESCE(yo.NachhaltigkeitCSRESG, 0.0) AS NachhaltigkeitCSRESG,
    COALESCE(yo.MarkeEnBW, 0.0) AS MarkeEnBW,
    COALESCE(yo.CLevelErwaehnungen, 0) AS CLevelErwaehnungen,

       -- =============================
    -- 5. Organische Kennzahlen
    -- =============================
    CASE
        WHEN (yo.VideoID IS NOT NULL AND yp.OrganicID IS NOT NULL) 
             AND (CAST(All_TotalViews AS BIGINT) - COALESCE(Paid_TotalViews, 0)) > 0
        THEN CAST(All_TotalViews AS BIGINT) - COALESCE(Paid_TotalViews, 0)
        ELSE CAST(All_TotalViews AS BIGINT)
    END AS Organic_TotalViews,

    CASE
        WHEN (yo.VideoID IS NOT NULL AND yp.OrganicID IS NOT NULL) 
             AND (CAST(All_TotalEngagements AS BIGINT) - COALESCE(Paid_TotalEngagements, 0)) > 0
        THEN CAST(All_TotalEngagements AS BIGINT) - COALESCE(Paid_TotalEngagements, 0)
        ELSE CAST(All_TotalEngagements AS BIGINT)
    END AS Organic_TotalEngagements

    -- COALESCE(yo.TotalWeightedEngagements, 0) AS Organic_TotalWeightedEngagements,
    -- COALESCE(yo.EngagementRateInPercent, 0) AS Organic_TotalEngagementRateInPercent,
    -- COALESCE(yo.TotalLikes, 0) AS Organic_TotalLikes,
    -- COALESCE(yo.TotalDislikes, 0) AS Organic_TotalDislikes,
    -- COALESCE(yo.TotalComments, 0) AS Organic_TotalComments,
    -- COALESCE(yo.TotalShares, 0) AS Organic_TotalShares,
    -- COALESCE(yo.TotalAverageViewDuration, 0) AS Organic_AverageViewDuration

FROM datif_pz_uk_{env}.03_transformed.youtube_organic_post_total AS yo
FULL OUTER JOIN datif_pz_uk_{env}.03_transformed.youtube_paid_total AS yp
    ON yo.VideoID = yp.OrganicID
""")
